# Face Identification using Computer Vision

In [29]:
# importing libraries
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from keras.models import load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [30]:

lando_norris_folder = r"dataset\Lando Norris"
other_drivers_folder = r"dataset/Other Drivers"
clean_data_Lando = r"clean data\Lando Norris"
clean_data_Other = r"clean data/Other Drivers"

def detect_and_crop_faces(input_folder, output_folder):
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') # load cascade for face detection
    for filename in os.listdir(input_folder):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

            for i, (x, y, w, h) in enumerate(faces):
                face = img[y:y+h, x:x+w]
                resized_face = cv2.resize(face, (160, 160))  # Resize to Facenet input size
                output_path = os.path.join(output_folder, f"{filename.split('.')[0]}_{i}.jpg")
                cv2.imwrite(output_path, resized_face)





In [31]:
# detect faces from norris and other drivers

detect_and_crop_faces(lando_norris_folder, clean_data_Lando)
detect_and_crop_faces(other_drivers_folder, clean_data_Other)

### Loading and pre-processing images

In [32]:


# initialize empty list for data and labels
data = []
labels = []

# function to load and pre-process data
def load_images(folder, label):
    for filename in os.listdir(folder):
        image_path = os.path.join(folder, filename) # path to an individual image
        image = cv2.imread(image_path) # read image
        image = cv2.resize(image, (100, 100)) # resize image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale

        # append image data and label
        data.append(image)
        labels.append(label)

# load data for both classes (Lando Norris and Other Drivers)
load_images(clean_data_Lando, 0) # label "0" for Lando Norris
load_images(clean_data_Other, 1) # label "1" for Other Drivers



In [33]:
# convert data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

### Split data into training and testing sets

In [34]:
# splitting train test data into 80/20 ratio
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

### CNN

In [35]:
# defining the cnn model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])


In [36]:
# compile the model
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
# train model
cnn_model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 3.4992 - accuracy: 0.4167 - val_loss: 420.6503 - val_accuracy: 0.5714
Epoch 2/10
1/1 [==============================] - 0s 159ms/step - loss: 416.1649 - accuracy: 0.5833 - val_loss: 202.0863 - val_accuracy: 0.5714
Epoch 3/10
1/1 [==============================] - 0s 158ms/step - loss: 197.2183 - accuracy: 0.5833 - val_loss: 11.2036 - val_accuracy: 0.5714
Epoch 4/10
1/1 [==============================] - 0s 150ms/step - loss: 6.4525 - accuracy: 0.6250 - val_loss: 98.6112 - val_accuracy: 0.4286
Epoch 5/10
1/1 [==============================] - 0s 147ms/step - loss: 92.9735 - accuracy: 0.4167 - val_loss: 81.3257 - val_accuracy: 0.4286
Epoch 6/10
1/1 [==============================] - 0s 151ms/step - loss: 77.7612 - accuracy: 0.4167 - val_loss: 45.4599 - val_accuracy: 0.4286
Epoch 7/10
1/1 [==============================] - 0s 148ms/step - loss: 43.1350 - accuracy: 0.4167 - val_loss: 18.5327 - val_accuracy: 0.4286
Epoch 8

In [38]:
# evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

1/1 [==============================] - 0s 34ms/step - loss: 8.9871 - accuracy: 0.3750
Test Accuracy: 37.50%


In [42]:
# make prediction
def predict_image(model, image_path, img_size=(100, 100)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, img_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #img = img.flatten()
    img = img / 255.0 # normalize pixel values
    img = np.expand_dims(img, axis=-1) #channel dimension for grayscale
    img = np.expand_dims(img, axis=0) # adds batch dimension


    prediction = cnn_model.predict(img)
    return "Lando Norris" if prediction < 0.5 else "Other Drivers"

result = predict_image(cnn_model, r"clean data\Lando Norris\20_0.jpg")
print(f"Prediction: {result}")

1/1 [==============================] - 0s 67ms/step
Prediction: Lando Norris
